# Importing and Connection to the database

In [20]:
import pymongo
import pprint
from math import dist

In [21]:
client = pymongo.MongoClient('localhost', 27017)
db = client['demo']
restaurants = db['restaurants']

# Basic Queries

### Get the first record from the database

In [22]:
query = restaurants.find_one()
pprint.pprint(query)

{'_id': ObjectId('63f66e984a7faf43c7575c88'),
 'address': {'building': '2780',
             'coord': [-73.98241999999999, 40.579505],
             'street': 'Stillwell Avenue',
             'zipcode': '11224'},
 'borough': 'Brooklyn',
 'cuisine': 'American',
 'grades': [{'date': datetime.datetime(2014, 6, 10, 0, 0),
             'grade': 'A',
             'score': 5},
            {'date': datetime.datetime(2013, 6, 5, 0, 0),
             'grade': 'A',
             'score': 7},
            {'date': datetime.datetime(2012, 4, 13, 0, 0),
             'grade': 'A',
             'score': 12},
            {'date': datetime.datetime(2011, 10, 12, 0, 0),
             'grade': 'A',
             'score': 12}],
 'name': 'Riviera Caterer',
 'restaurant_id': '40356018'}


### Get the information for the restaurant called "Towne Cafe"

In [23]:
query = restaurants.find({"name": "Towne Cafe"})
for restaurant in query:
    pprint.pprint(restaurant)

{'_id': ObjectId('63f66e984a7faf43c7575cd5'),
 'address': {'building': '2602',
             'coord': [-73.95443709999999, 40.5877993],
             'street': 'East   15 Street',
             'zipcode': '11235'},
 'borough': 'Brooklyn',
 'cuisine': 'American',
 'grades': [{'date': datetime.datetime(2014, 5, 14, 0, 0),
             'grade': 'A',
             'score': 11},
            {'date': datetime.datetime(2013, 4, 27, 0, 0),
             'grade': 'A',
             'score': 9},
            {'date': datetime.datetime(2012, 11, 23, 0, 0),
             'grade': 'B',
             'score': 27},
            {'date': datetime.datetime(2012, 3, 14, 0, 0),
             'grade': 'B',
             'score': 17},
            {'date': datetime.datetime(2011, 7, 14, 0, 0),
             'grade': 'B',
             'score': 21}],
 'name': 'Towne Cafe',
 'restaurant_id': '40364681'}


### What Restaurants are on Church Street

In [52]:
query = restaurants.find({"address.street":"Church Street"}, {"name":1,"_id":0})
for i in range(10):
    pprint.pprint(query[i])

{'name': 'Downtown Deli'}
{'name': "Domino'S Pizza"}
{'name': "South'S"}
{'name': 'Macao  Restuarant And Bar'}
{'name': 'B Flat'}
{'name': 'La Colombe Torrefaction'}
{'name': "Saluggi'S"}
{'name': 'Westside Coffee Shop Restaurant'}
{'name': 'Dunkin Donuts'}
{'name': 'Starbucks Coffee'}


# Level 2 Queries

# Find number of restaurants in Brooklyn

In [25]:
query1 = restaurants.count_documents({"borough":"Brooklyn"})

pprint.pprint(query1)


6086


# Find number of Italian Restaurants in Bronx

In [26]:
query2 = restaurants.count_documents({"cuisine":"Italian", "borough":"Bronx"})

pprint.pprint(query2)

52


# Level 3 Queries

# How many Restaurants are in each Zipcode

In [51]:
query3 = restaurants.aggregate([
    {"$group": {"_id": "$address.zipcode","num": {"$sum": 1}}},
    {"$sort": {"num":-1}} 
])

query_list = list(query3)

for q in range(10):
    pprint.pprint(query_list[q])
    

{'_id': '10003', 'num': 686}
{'_id': '10019', 'num': 675}
{'_id': '10036', 'num': 611}
{'_id': '10001', 'num': 520}
{'_id': '10022', 'num': 485}
{'_id': '10013', 'num': 480}
{'_id': '10002', 'num': 471}
{'_id': '10011', 'num': 467}
{'_id': '10016', 'num': 433}
{'_id': '10014', 'num': 428}


# How many Restaurants per cuisine

In [50]:
query4 = restaurants.aggregate([
    {"$group": {"_id": "$cuisine","num": {"$sum": 1}}},
    {"$sort": {"num":-1}} 
])

query_list = list(query4)

for i in range(10):
    pprint.pprint(query_list[i])

{'_id': 'American', 'num': 6183}
{'_id': 'Chinese', 'num': 2418}
{'_id': 'Café/Coffee/Tea', 'num': 1214}
{'_id': 'Pizza', 'num': 1163}
{'_id': 'Italian', 'num': 1069}
{'_id': 'Other', 'num': 1011}
{'_id': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
 'num': 850}
{'_id': 'Japanese', 'num': 760}
{'_id': 'Mexican', 'num': 754}
{'_id': 'Bakery', 'num': 691}


# Level 4 Queries

# Using Match Stage to Filter for boroughs with over 1,000 restaurants

In [29]:
query5 = restaurants.aggregate([
    {"$group": {"_id": "$borough","num": {"$sum": 1}}},
    {'$match': {'num' : {'$gte' : 1000}}},
    {"$sort": {"num":-1}} 
])

query_list = list(query5)

for q in query_list:
    pprint.pprint(q)


{'_id': 'Manhattan', 'num': 10259}
{'_id': 'Brooklyn', 'num': 6086}
{'_id': 'Queens', 'num': 5656}
{'_id': 'Bronx', 'num': 2338}


# Using Match to filter for Irish Restaurants in Manhattan

In [30]:
query6 = restaurants.aggregate([
    {'$match': {'cuisine' : {'$eq' : 'Irish'}, 'borough': {'$eq' : 'Manhattan'}}},
    {"$group": {"_id": "$cuisine", "num": {"$sum": 1}}},
    {"$sort": {"num":-1}} 
])

query_list = list(query6)

for q in query_list:
    pprint.pprint(q)


{'_id': 'Irish', 'num': 116}


# Level 5 Queries

# Finding 5 closest restaurants to empire state building
- Add field for distance from empire state
- Return 5 closest distances

In [31]:
empire_state_coord = [-73.9851304, 40.748817]

pipeline =[
  {
    '$addFields': {
      'distance': {
        '$sqrt': {
          '$add': [
            {
              '$pow': [
                {
                  '$subtract': [
                    {
                      '$arrayElemAt': [
                        '$address.coord', 0
                      ]
                    }, empire_state_coord[0]
                  ]
                }, 2
              ]
            }, {
              '$pow': [
                {
                  '$subtract': [
                    {
                      '$arrayElemAt': [
                        '$address.coord', 1
                      ]
                    }, empire_state_coord[1]
                  ]
                }, 2
              ]
            }
          ]
        }
      }
    }
  },
{'$project': {'_id' : 0, 'name': 1, 'distance': 1}},
{"$sort": {"distance":1}} 
]


Citation:
MongoDB. (2019, February 5). How to calculate distance between two geolocation points [Online forum post].
MongoDB Community Forums. 
https://www.mongodb.com/community/forums/t/how-to-calculate-distance-between-two-geolocation-points/173045/2

In [32]:
query7 = restaurants.aggregate(pipeline)

query_list = list(query7)

for i in range(5):
    pprint.pprint(query_list[i])

{'distance': None, 'name': "Fratelli'S Market Place"}
{'distance': None, 'name': 'Eugene & Co'}
{'distance': 0.0004539718163053264, 'name': 'Subway'}
{'distance': 0.0004539718163053264, 'name': 'The Stellan'}
{'distance': 0.0005296839151894624, 'name': 'Potbelly Sandwich Shop'}


# Finding 5 Best Indian Restaurants in Manhattan

- Finding american restaurants in manhattan
- Adding Average score field
- Returning 5 best scores (lower score is better)

In [42]:
# Number of Indian Restaurants in Manhattan
query6 = restaurants.aggregate([
    {'$match': {'cuisine' : {'$eq' : 'Indian'}, 'borough': {'$eq' : 'Manhattan'}}},
    {"$group": {"_id": "$cuisine", "num": {"$sum": 1}}},
    {"$sort": {"num":-1}} 
])

query_list = list(query6)

for q in query_list:
    pprint.pprint(q)


{'_id': 'Indian', 'num': 152}


In [49]:
# adding average score fields for each restaurant
query6 = restaurants.aggregate([
    {'$match': {'cuisine': {'$eq': 'Indian'},'borough': {'$eq': 'Manhattan'}}},
    {'$project': {'_id': 0,'name': 1,'borough': 1, 'cuisine': 1,
                  'num_grades': {'$size': '$grades'},'total_score': {'$sum': '$grades.score'}}},
    {'$match': {'num_grades': {'$gt': 0}}},
    {'$addFields': {'average_score': {'$divide': ['$total_score', '$num_grades']}}},
    {'$sort': {'average_score': 1}}])

query_list = list(query6)

for i in range(5):
    pprint.pprint(query_list[i])


{'average_score': 2.5,
 'borough': 'Manhattan',
 'cuisine': 'Indian',
 'name': 'Diwan-E-Khaas',
 'num_grades': 4,
 'total_score': 10}
{'average_score': 3.0,
 'borough': 'Manhattan',
 'cuisine': 'Indian',
 'name': 'Bricklane Curry House',
 'num_grades': 1,
 'total_score': 3}
{'average_score': 4.0,
 'borough': 'Manhattan',
 'cuisine': 'Indian',
 'name': 'Soho Tiffin Junction',
 'num_grades': 1,
 'total_score': 4}
{'average_score': 4.75,
 'borough': 'Manhattan',
 'cuisine': 'Indian',
 'name': 'Diwan-E- Khaas',
 'num_grades': 4,
 'total_score': 19}
{'average_score': 5.75,
 'borough': 'Manhattan',
 'cuisine': 'Indian',
 'name': 'Taj Cafe',
 'num_grades': 4,
 'total_score': 23}
